<a href="https://colab.research.google.com/github/Jordy-20035/hw_1/blob/main/%D0%9A%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%86%D0%B8%D1%8F_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%BE%D0%B2_%D0%BF%D0%BE_%D1%82%D0%BE%D0%BF%D0%B8%D0%BA%D0%B0%D0%BC_%D1%81_%D0%B8%D1%81%D0%BF%D0%BE%D0%BB%D1%8C%D0%B7%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5%D0%BC_TF_IDFx2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install corus py imbalanced-learn pymorphy3 pymorphy3-dicts-ru joblib tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 kB 398.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 39.2 MB/s eta 0:00:00


In [5]:
import os
print(os.path.exists('lenta-ru-news.csv.gz'))  # Должно вернуть True

False


In [7]:
# Импорт необходимых библиотек
import pandas as pd
from corus import load_lenta
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
import nltk
from pymorphy3 import MorphAnalyzer
import re
import random

# Инициализация рандом-стейта для воспроизводимости
random_state = 42
random.seed(random_state)

# Загрузка данных
def load_data(limit=100000):
    # Скачивание файла, если его нет
    if not os.path.exists('lenta-ru-news.csv.gz'):
      !wget -q https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

    records = list(load_lenta('lenta-ru-news.csv.gz'))[:limit]
    data = []
    for record in records:
        data.append({
            'title': record.title,
            'text': record.text,
            'topic': record.topic
        })
    return pd.DataFrame(data)

# Загрузка и фильтрация данных
df = load_data()
df = df.dropna(subset=['topic', 'text', 'title']).reset_index(drop=True)

# Предобработка текста
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))
morph = MorphAnalyzer()

def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^а-яё\s]', '', text)
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words and len(word) > 2]
    lemmas = [morph.parse(word)[0].normal_form for word in tokens]
    return ' '.join(lemmas)

# Объединение title и text
df['full_text'] = df['title'] + ' ' + df['text']
df['processed_text'] = df['full_text'].apply(preprocess)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [8]:
df['topic'].value_counts().sort_values().tail(10)

,count
topic,
Из жизни,4903
Наука и техника,5645
Культура,6578
Бывший СССР,6810
Силовые структуры,6925
Интернет и СМИ,6935
Экономика,7682
Спорт,10045
Мир,14421


In [9]:
# Оставляем только классы с >= 10 примерами
min_samples = 10
topic_counts = df['topic'].value_counts()
valid_topics = topic_counts[topic_counts >= min_samples].index
df_filtered = df[df['topic'].isin(valid_topics)]

# Проверка
print(f"Удалено классов: {len(topic_counts) - len(valid_topics)}")

Удалено классов: 1


In [10]:
# Замените оригинальный df на df_filtered
X_train, X_temp, y_train, y_temp = train_test_split(
    df_filtered['processed_text'],
    df_filtered['topic'],
    test_size=0.4,
    random_state=random_state,
    stratify=df_filtered['topic']
)

In [11]:
# Разделение данных с учетом стратификации
# X_train, X_temp, y_train, y_temp = train_test_split(
#     df['processed_text'], df['topic'],
#     test_size=0.4, random_state=random_state, stratify=df['topic']
# )

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.5, random_state=random_state, stratify=y_temp
)

# Бейзлайн модель
dummy = DummyClassifier(strategy='most_frequent', random_state=random_state)
dummy.fit(X_train, y_train)
dummy_pred = dummy.predict(X_val)
print(f'Dummy accuracy: {accuracy_score(y_val, dummy_pred):.4f}')

# Создание пайплайнов для векторизации и обучения
def grid_search(vectorizer, X_train, y_train):
    pipe = Pipeline([
        ('vec', vectorizer),
        ('clf', LogisticRegression(random_state=random_state, max_iter=1000))
    ])

    param_grid = {
        'vec__max_features': [10000, 50000],
        'vec__ngram_range': [(1,1), (1,2)],
        'clf__C': [0.1, 1, 10]
    }

    grid = GridSearchCV(pipe, param_grid, cv=3, scoring='accuracy', n_jobs=-1)
    grid.fit(X_train, y_train)
    return grid

# Поиск лучших параметров для CountVectorizer
count_grid = grid_search(CountVectorizer(), X_train, y_train)
print(f'Best CountVectorizer parameters: {count_grid.best_params_}')
print(f'Validation accuracy: {count_grid.score(X_val, y_val):.4f}')

# Поиск лучших параметров для TfidfVectorizer
tfidf_grid = grid_search(TfidfVectorizer(), X_train, y_train)
print(f'Best TfidfVectorizer parameters: {tfidf_grid.best_params_}')
print(f'Validation accuracy: {tfidf_grid.score(X_val, y_val):.4f}')

# Оценка лучшей модели на тестовой выборке
best_model = tfidf_grid if tfidf_grid.best_score_ > count_grid.best_score_ else count_grid
test_accuracy = best_model.score(X_test, y_test)
print(f'Test accuracy of best model: {test_accuracy:.4f}')

Dummy accuracy: 0.1515


NameError: name 'Pipeline' is not defined